<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/02_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# finalTable
## Table of Content

1. [Import packages](#packages)
2. [Import data](#data)
3. [Data Cleaning](#cleaning)
    1. [Cleaning damage data](#damage)
    2. [Cleaning contract data](#contract)
4. [Joining Damage and Contract](#join)
5. [Risk Score Definition](#riskscore)

## Import packages <a id="packages"></a>

In [1]:
# Install necessary libraries
!pip install pandas openpyxl
!pip install numpy
!pip install plotly

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
# from google.colab import drive

In [2]:
# Ignore warnings
warnings.filterwarnings('ignore')

## Import data <a id="data"></a>

In [4]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
# Define paths to the CSV files
contractPath = '/content/drive/My Drive/Capstone/Data/combined_contracts.csv'
damagePath = '/content/drive/My Drive/Capstone/Data/damages.csv'

In [7]:
# Load the CSV files into DataFrames in chunks
chunk_size = 10000  # Adjust based on your memory limitations

contract_chunks = pd.read_csv(contractPath, chunksize=chunk_size)
contract_all = pd.concat(contract_chunks)

damage_chunks = pd.read_csv(damagePath, chunksize=chunk_size)
damage = pd.concat(damage_chunks)

In [35]:
# Import data form local machine
damagePath = '../../data/damage.csv'
damage = pd.read_csv(damagePath)

contractPath = '../../data/combined_contracts.csv'
contract_all = pd.read_csv(contractPath)

In [36]:
print(len(contract_all))
print(len(damage))

11534325
425738


In [37]:
print('Number of unique values:')
contract_all.nunique()

Number of unique values:


ANO_SID                         1683447
CORPORATE_DEVISION                    4
ORTPLZ                            18231
ORTS-NAME                         15910
STRASSE                          222182
SUM_INSURED                      512709
CONSTRACTION_DESIGN                  13
CONSTRUCTION_YEAR                   556
BASEMENT                              2
BUILDING_TYPE                        11
WFL                                 943
ZONE                                 23
SF-SYSTEM                             5
TYPE_OF_DEDUCTIBLE                    5
Selbstbeteiligung in Prozent         12
SBMIN                               105
DEDUCTIBLE_MAX                      106
DRAIN_PIPE_INSURED                    1
PRODUCTLINE                           6
INVENTORY_MANAGEMENT_SYSTEM           2
KEYDATE                              11
PRIOR_DAMAGES                        10
RISK_NATION                           1
UVV-KZ                                2
SFR-JAHRE                            33


In [39]:
print('Number of observations per unique value:')
print(contract_all['CORPORATE_DEVISION'].value_counts())
print(contract_all['TYPE_OF_DEDUCTIBLE'].value_counts())

Number of observations per unique value:
CORPORATE_DEVISION
VHV    7071068
VGV    3195545
W&W     678568
H&H     589144
Name: count, dtype: int64
TYPE_OF_DEDUCTIBLE
0    11503419
3       25203
4        3584
5        2111
2           8
Name: count, dtype: int64


# Data Cleaning <a id="cleaning"></a>

## Cleaning damage data <a id="damage"></a>

Drop these variables:
- The vast majority of data in ADDRESS_TYPE, DAMAGE_REPORTER, DATE_COMPLETION_DATE and CUMULEVENT are missing.
- Exclude variables with only one (or very few/not relevant) category: 'DEPARTMENT_SEGMENT', 'DEPARTMENT_RESPONSIBLE', 'DEPARTMENT_CUSTOMER_SEGMENT', 'BUSINESS_TYPE_MARK', 'DAMAGE_BRANCH_DESIGNMENT', 'DAMAGE_BRANCH', 'SVCLUSTER', ‘BUSINESS_MARK’
- Not relevant: 'PARTY_ID', 'RESTORATION_CANDIDAT'

In [12]:
# Drop columns
columnsToDrop = [
    'ABTEILG_LVL01_SEGMENT', 'ABTEILG_LVL02_VERANTWORTUNG', 'ABTEILG_LVL03_KD_SEGMENT', 'GESCHAEFTSTYP_KZ', 'GESCHAEFTS_KZ',
    'DAMAGE_TRAIT_DESIGNMENT', 'DAMAGE_TYPE', 'VER_SVCLUSTER_KZ', 'ADDRESS_TYPE', 'AGT_SCHADENMELDER', 'DAT_ERLFDAT_ERLEDIGUNG_FORD',
    'PARTY_ID', 'CUMULEVENT', 'SANIERUNGSKANDIDAT_KZ']

damage = damage.drop(columns=columnsToDrop, errors='ignore')

In [14]:
# Convert dates to date format
damage['DAT_EREIGNIS'] = pd.to_datetime(damage['DAT_EREIGNIS'], errors='coerce', infer_datetime_format=True)
damage['DAT_ANL_ERSTRESERVIERUNG'] = pd.to_datetime(damage['DAT_ANL_ERSTRESERVIERUNG'], errors='coerce', infer_datetime_format=True, dayfirst=True)

# Convert 'TOTAL_DAMAGE' to float
damage['TOTAL_DAMAGE'] = damage['TOTAL_DAMAGE'].replace('UNDEFINED', np.nan).astype('float64')

# Convert ids to object
damage['GESCHAEFTS_FELD_ID'] = damage['GESCHAEFTS_FELD_ID'].astype('object')
damage['ANO_SID'] = damage['ANO_SID'].astype('object')
damage['SCH_MERKMAL_KZ'] = damage['SCH_MERKMAL_KZ'].astype('object')
damage['SCH_URSACHE_KZ'] = damage['SCH_URSACHE_KZ'].astype('object')
damage['DAMAGE_CLUSTER_ID'] = damage['DAMAGE_CLUSTER_ID'].astype('object')
damage['VER_STATNR'] = damage['VER_STATNR'].astype('object')

In [17]:
# Drop observations outside of Germany
damage = damage[damage['SCHADEN_LKZ'] == 'D']

# Drop column 'SCHADEN_LKZ'
damage = damage.drop(columns=['SCHADEN_LKZ'], errors='ignore')

# Display the length of the DataFrame
len(damage)

323252

In [18]:
# Drop observations before 2014
damage = damage[damage['DAT_EREIGNIS'].dt.year >= 2014]
damage = damage[damage['DAT_ANL_ERSTRESERVIERUNG'].dt.year >= 2014]

# Display the length of the DataFrame
len(damage)

261285

Drop negative values from financial variables:

In [20]:
# Check number of negative values in financial columns
financialColumns = ['EXPENSE', 'INITIAL_RESERVATION', 'TOTAL_DAMAGE', 'RESIDUAL_AMOUNT', 'PAYOUT']

# Convert financial columns to numeric values
for column in financialColumns:
    damage[column] = pd.to_numeric(damage[column], errors='coerce')

# Create a dictionary
negativeCounts = {}

# Count negative values in each column
for column in financialColumns:
    negativeCounts[column] = (damage[column] < 0).sum()

print(negativeCounts)

# Drop rows with negative values in financial columns
for column in financialColumns:
    damage = damage[damage[column] >= 0]

# Display the length of the DataFrame after dropping negative values
print(len(damage))


{'EXPENSE': 90, 'INITIAL_RESERVATION': 0, 'TOTAL_DAMAGE': 0, 'RESIDUAL_AMOUNT': 348, 'PAYOUT': 93}
205997


In [22]:
# Define the financial columns
financialColumns = ['EXPENSE', 'INITIAL_RESERVATION', 'TOTAL_DAMAGE', 'RESIDUAL_AMOUNT', 'PAYOUT']

# Create a mask that is True for rows where all financial columns are >= 0 or NaN
mask = (damage[financialColumns] >= 0) | damage[financialColumns].isna()

# Use .all(axis=1) to ensure that we only drop rows where all conditions are False (i.e., where all are negative)
damage = damage[mask.all(axis=1)]

# Display the length of the DataFrame after applying the mask
print(len(damage))

205997


In [24]:
# Convert zip codes starting with 'W' or 'O', or those not 5 characters long to NaN
damage.loc[damage['DAMAGE_EVENT_ZIP_CODE'].str.startswith(('W', 'O')) |
           (damage['DAMAGE_EVENT_ZIP_CODE'].str.len() != 5), 'DAMAGE_EVENT_ZIP_CODE'] = np.nan

In [26]:
# Replace "Quartalsabrechnung" or "Monatsabrechnung" with NaN
damage['DAMAGE_EVENT_STREET'] = damage['DAMAGE_EVENT_STREET'].replace(['Quartalsabrechnung', 'Monatsabrechnung'], np.nan)

In [13]:
# Save dataset to csv
damage.to_csv('../../data/damage_clean.csv', index=False)

In [14]:
# Exclude rows where EXPENSE, PAYOUT, and RESIDUAL_AMOUNT are all zero
# damage = damage[(damage['EXPENSE'] != 0) | (damage['PAYOUT'] != 0) | (damage['RESIDUAL_AMOUNT'] != 0)]
# print(len(damage))

## Cleaning contract data <a id="contract"></a>

Drop these variables:
- The majority of data is missing for the variables BASEMENT, BUILDING_TYPE, NO_CLAIMS_SCALE, NO_CLAIMS_YEARS.
- Not relevant for our analysis: 'TYPE_OF_DEDUCTIBLE', 'DEDUCTIBLE_PERCENTAGE', 'DEDUCTIBLE_MIN', 'DEDUCTIBLE_MAX', 'INVENTORY_MANAGEMENT_SYSTEM', 'PARTY-ID', 'RISK_NATION', 'Day', 'Month'

In [40]:
# Define the columns to drop with updated variable names
columnsToDrop = [
    'Selbstbeteiligung in Prozent', 'SBMIN', 'DEDUCTIBLE_MAX', 'INVENTORY_MANAGEMENT_SYSTEM',
    'BASEMENT', 'BUILDING_TYPE', 'NO_CLAIMS_SCALE', 'SFR-JAHRE', 'RISK_NATION', 'PARTY_ID', 'KEYDATE'
]

# Drop columns from the DataFrame
contract_all = contract_all.drop(columns=columnsToDrop, errors='ignore')

In [41]:
# Convert 'contract_year' to date format if it represents a date or use another date-related column
# Since 'contract_year' appears to be an integer or year, we'll create a date column for demonstration purposes
contract_all['contract_year'] = pd.to_datetime(contract_all['contract_year'], format='%Y', errors='coerce')

# Convert ids to object
contract_all['ANO_SID'] = contract_all['ANO_SID'].astype('object')

# Replace '     .' with NaN in PREMIUM_AMOUNT
contract_all['PREMIUM_AMOUNT'] = contract_all['PREMIUM_AMOUNT'].replace('     .', np.nan)
# Convert PREMIUM_AMOUNT to float
contract_all['PREMIUM_AMOUNT'] = contract_all['PREMIUM_AMOUNT'].astype(float)

# Replace '     .' with NaN in SUM_INSURED
contract_all['SUM_INSURED'] = contract_all['SUM_INSURED'].replace('     .', np.nan)
# Convert SUM_INSURED to float
contract_all['SUM_INSURED'] = contract_all['SUM_INSURED'].astype(float)

Drop negative values from financial variables (there are none):

In [42]:
# Check number of negative values in financial columns
financialColumns = ['SUM_INSURED', 'PREMIUM_AMOUNT']

# Create a dictionary
negativeCounts = {}

# Count negative values in each column
for column in financialColumns:
    negativeCounts[column] = (contract_all[column] < 0).sum()

print(negativeCounts) # nothing to drop

{'SUM_INSURED': 0, 'PREMIUM_AMOUNT': 0}


In [43]:
# Display column names to check for correct names
print(contract_all.columns)

# Convert zip codes starting with 'W' or 'O', or those not 5 characters long to NaN
contract_all.loc[contract_all['ORTPLZ'].str.startswith(('W', 'O')) |
            (contract_all['ORTPLZ'].str.len() != 5), 'ORTPLZ'] = np.nan

len(contract_all)

Index(['ANO_SID', 'CORPORATE_DEVISION', 'ORTPLZ', 'ORTS-NAME', 'STRASSE',
       'SUM_INSURED', 'CONSTRACTION_DESIGN', 'CONSTRUCTION_YEAR', 'WFL',
       'ZONE', 'SF-SYSTEM', 'TYPE_OF_DEDUCTIBLE', 'DRAIN_PIPE_INSURED',
       'PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER', 'PARTY-ID',
       'PREMIUM_AMOUNT', 'contract_year'],
      dtype='object')


11534325

In [45]:
# Replace living space = 0 with average

# Calculate the average of WFL, excluding zeros
non_zero_living_space = contract_all[contract_all['WFL'] != 0]['WFL']
avg_living_space = non_zero_living_space.mean()

# Replace zero values with the calculated average
contract_all.loc[contract_all['WFL'] == 0, 'WFL'] = avg_living_space

len(contract_all)

11534325

In [46]:
# Replace uncommon values for the construction year with the average
currentYear = 2024

# Calculate the average of CONSTRUCTION_YEAR, excluding future years and specific unwanted values
validYears = contract_all[
    (contract_all['CONSTRUCTION_YEAR'] <= currentYear) &
    (~contract_all['CONSTRUCTION_YEAR'].isin([1111, 1, 0]))
]['CONSTRUCTION_YEAR']
avgYear = validYears.mean()

# Replace unwanted values with the calculated average
contract_all.loc[
    (contract_all['CONSTRUCTION_YEAR'] > currentYear) |
    (contract_all['CONSTRUCTION_YEAR'].isin([1111, 1, 0])), 'CONSTRUCTION_YEAR'
] = avgYear

len(contract_all)

11534325

In [48]:
# Replace values in the 'DRAIN_PIPE_INSURED' column
contract_all['DRAIN_PIPE_INSURED'] = contract_all['DRAIN_PIPE_INSURED'].replace({'J': 1, 'N': 0})

# Replace values in the 'UVV-KZ' column
contract_all['UVV-KZ'] = contract_all['UVV-KZ'].replace({'J': 1, 'N': 0})

len(contract_all)

11534325

In [49]:
print('Number of observations per unique value:')
print(contract_all['CORPORATE_DEVISION'].value_counts())
print(contract_all['TYPE_OF_DEDUCTIBLE'].value_counts())

Number of observations per unique value:
CORPORATE_DEVISION
VHV    7071068
VGV    3195545
W&W     678568
H&H     589144
Name: count, dtype: int64
TYPE_OF_DEDUCTIBLE
0    11503419
3       25203
4        3584
5        2111
2           8
Name: count, dtype: int64


In [24]:
# Save dataset to csv
contract_all.to_csv('../all_data_cleaned.csv', index=False)

In [34]:
%store damage

Stored 'damage' (DataFrame)


In [35]:
%store contract_all

Stored 'contract_all' (DataFrame)


# Joining Damage and Contract <a id="join"></a>

In [37]:
# Extract year into new column
damage['Year'] = damage['DAT_ANL_ERSTRESERVIERUNG'].dt.year

# Display the first few rows to verify changes
print(damage.head())

      EXPENSE     DAMAGE_CLUSTER_ID DAT_ANL_ERSTRESERVIERUNG DAT_EREIGNIS  \
1818   333.20  0000014351b801943000               2014-02-01   2014-01-01   
1819  1835.35  00000143524468b42000               2014-08-01   2014-01-02   
1820  1182.23  0000014352bf46843000               2014-03-01   2014-01-01   
1827   800.04  0000014367323354a000               2014-10-01   2014-01-05   
1875  1704.10  00000143d3c6ac54d000               2014-03-02   2014-01-09   

       ANO_SID  INITIAL_RESERVATION  TOTAL_DAMAGE GESCHAEFTS_FELD_ID  \
1818  43597657                500.0        1000.0                  0   
1819  29261516                500.0         500.0                  0   
1820   4118112               3500.0        3500.0                  0   
1827  41087246                500.0         500.0                  0   
1875  32662333               1850.0        1835.0                  0   

      RESIDUAL_AMOUNT DAMAGE_INCIDENT  ... ANO_SID.1 DIVISION_LEVEL_00  \
1818              0.0    Water

In [41]:
# Ensure ANO_SID is of the same type in both DataFrames
damage['ANO_SID'] = damage['ANO_SID'].astype('object')
contract_all['ANO_SID'] = contract_all['ANO_SID'].astype('object')

# Extract the year from 'contract_year' to match 'Year'
contract_all['Year'] = contract_all['contract_year'].dt.year

# Combine Damage and Contract Data
damage_all = pd.merge(damage, contract_all, left_on=["ANO_SID", "Year"], right_on=["ANO_SID", "Year"], how='left')

print(len(damage))
print(len(damage_all))

# Display the first few rows to verify changes
print(damage_all.head())

39902
40665
   EXPENSE     DAMAGE_CLUSTER_ID DAT_ANL_ERSTRESERVIERUNG DAT_EREIGNIS  \
0   333.20  0000014351b801943000               2014-02-01   2014-01-01   
1  1835.35  00000143524468b42000               2014-08-01   2014-01-02   
2  1182.23  0000014352bf46843000               2014-03-01   2014-01-01   
3   800.04  0000014367323354a000               2014-10-01   2014-01-05   
4  1704.10  00000143d3c6ac54d000               2014-03-02   2014-01-09   

    ANO_SID  INITIAL_RESERVATION  TOTAL_DAMAGE GESCHAEFTS_FELD_ID  \
0  43597657                500.0        1000.0                  0   
1  29261516                500.0         500.0                  0   
2   4118112               3500.0        3500.0                  0   
3  41087246                500.0         500.0                  0   
4  32662333               1850.0        1835.0                  0   

   RESIDUAL_AMOUNT DAMAGE_INCIDENT  ... SF-SYSTEM TYPE_OF_DEDUCTIBLE  \
0              0.0    Water damage  ...       0.0       

In [43]:
# Display column names to check for correct names
print(damage_all.columns)

# Define columns to drop
columns_to_drop = ["RISK_LOCATION_ZIP_CODE", "RISK_LOCATION_TOWN", "RISK_LOCATION_STREET"]

# Only drop columns that exist in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in damage_all.columns]

# Drop the columns
damage_all = damage_all.drop(columns=columns_to_drop, axis=1)

# Display the first few rows to verify changes
print(damage_all.head())

Index(['EXPENSE', 'DAMAGE_CLUSTER_ID', 'DAT_ANL_ERSTRESERVIERUNG',
       'DAT_EREIGNIS', 'ANO_SID', 'INITIAL_RESERVATION', 'TOTAL_DAMAGE',
       'GESCHAEFTS_FELD_ID', 'RESIDUAL_AMOUNT', 'DAMAGE_INCIDENT',
       'DAMAGE_EVENT_TOWN', 'DAMAGE_EVENT_ZIP_CODE', 'DAMAGE_EVENT_STREET',
       'SCH_MERKMAL_KZ', 'SCH_URSACHE_KZ', 'DAMAGE_CAUSE_DESIGNMENT',
       'SCH_ZW_BEZ', 'SCH_ZW_CD', 'ANO_SID.1', 'DIVISION_LEVEL_00',
       'DIVISION_LEVEL_01', 'DIVISION_LEVEL_02', 'DIVISION_LEVEL_03',
       'DIVISION_LEVEL_04', 'DIVISION_LEVEL_05', 'VER_STATNR', 'PAYOUT',
       'Year', 'CORPORATE_DEVISION', 'ORTPLZ', 'ORTS-NAME', 'STRASSE',
       'SUM_INSURED', 'CONSTRACTION_DESIGN', 'CONSTRUCTION_YEAR', 'WFL',
       'ZONE', 'SF-SYSTEM', 'TYPE_OF_DEDUCTIBLE', 'DRAIN_PIPE_INSURED',
       'PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER', 'PARTY-ID',
       'PREMIUM_AMOUNT', 'contract_year'],
      dtype='object')
   EXPENSE     DAMAGE_CLUSTER_ID DAT_ANL_ERSTRESERVIERUNG DAT_EREIGNIS  \
0   3

In [44]:
# Define columns to drop
columns_to_drop = ["DAMAGE_EVENT_ZIP_CODE", "DAMAGE_EVENT_TOWN", "DAMAGE_EVENT_STREET"]

# Drop the columns
damage_all = damage_all.drop(columns=columns_to_drop, axis=1)

# Display the first few rows to verify changes
print(damage_all.head())

   EXPENSE     DAMAGE_CLUSTER_ID DAT_ANL_ERSTRESERVIERUNG DAT_EREIGNIS  \
0   333.20  0000014351b801943000               2014-02-01   2014-01-01   
1  1835.35  00000143524468b42000               2014-08-01   2014-01-02   
2  1182.23  0000014352bf46843000               2014-03-01   2014-01-01   
3   800.04  0000014367323354a000               2014-10-01   2014-01-05   
4  1704.10  00000143d3c6ac54d000               2014-03-02   2014-01-09   

    ANO_SID  INITIAL_RESERVATION  TOTAL_DAMAGE GESCHAEFTS_FELD_ID  \
0  43597657                500.0        1000.0                  0   
1  29261516                500.0         500.0                  0   
2   4118112               3500.0        3500.0                  0   
3  41087246                500.0         500.0                  0   
4  32662333               1850.0        1835.0                  0   

   RESIDUAL_AMOUNT DAMAGE_INCIDENT  ... SF-SYSTEM TYPE_OF_DEDUCTIBLE  \
0              0.0    Water damage  ...       0.0                0.0

In [45]:
damage_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40665 entries, 0 to 40664
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   EXPENSE                   40665 non-null  float64       
 1   DAMAGE_CLUSTER_ID         40665 non-null  object        
 2   DAT_ANL_ERSTRESERVIERUNG  40665 non-null  datetime64[ns]
 3   DAT_EREIGNIS              40665 non-null  datetime64[ns]
 4   ANO_SID                   40665 non-null  object        
 5   INITIAL_RESERVATION       40665 non-null  float64       
 6   TOTAL_DAMAGE              40665 non-null  float64       
 7   GESCHAEFTS_FELD_ID        40665 non-null  object        
 8   RESIDUAL_AMOUNT           40665 non-null  float64       
 9   DAMAGE_INCIDENT           40665 non-null  object        
 10  SCH_MERKMAL_KZ            40665 non-null  object        
 11  SCH_URSACHE_KZ            40665 non-null  object        
 12  DAMAGE_CAUSE_DESIG

# Risk Score Definition <a id="riskscore"></a>

In [46]:
# Calculate NET_PREMIUM_AMOUNT as 60% of PREMIUM_AMOUNT
damage_all['NET_PREMIUM_AMOUNT'] = damage_all['PREMIUM_AMOUNT'] * 0.6

In [47]:
# Calculate PIPE_PREMIUM_AMOUNT based on CORPORATE_DEVISION
# Map devision percentages
devisionPercentages = {
    'VGV': 0.43,
    'VHV': 0.11,
    'H&H': 0.07,
    'W&W': 0.32
}

# Calculate PIPE_PREMIUM_AMOUNT
damage_all['PIPE_PREMIUM_AMOUNT'] = damage_all['DIVISION_LEVEL_02'].map(devisionPercentages) * damage_all['NET_PREMIUM_AMOUNT']

In [48]:
# Calculate RISK_SCORE as PIPE_PREMIUM_AMOUNT / EXPENSE
damage_all['RISK_SCORE'] = damage_all['EXPENSE'] / damage_all['PIPE_PREMIUM_AMOUNT']

# When expense = 0 the division returns 0. Instead return NaN.
# damage_all['RISK_SCORE'] = damage_all['RISK_SCORE'].replace(0, np.nan)

In [49]:
# Check for nas in RISK_SCORE
print(len(damage_all))
print(damage_all['RISK_SCORE'].isna().sum())
print(len(damage_all.dropna(subset=['RISK_SCORE'])))

40665
4964
35701


In [50]:
# Check nas in PREMIUM_AMOUNT and EXPENSE
print(damage_all['PREMIUM_AMOUNT'].isna().sum())
print(damage_all['EXPENSE'].isna().sum())

4927
0


In [58]:
# Save dataset to csv
damage_all.to_csv('../content/drive/My Drive/Capstone/Data/all_data_cleaned.csv', index=False)

In [56]:
%store damage_all

Stored 'damage_all' (DataFrame)


In [54]:
# %store -r damage
# %store -r contract_all
# %store -r damage_all